In [29]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

In [30]:
with open('data_20240723.txt', 'r') as f:
    data = f.read()

data_pattern = re.compile(r'(\d{4})\s+(\d{3})\s+(\d{1,2})\s+(\d{1,2})\s+(\d{3})\s+([-]?\d+)\s+(\d+)\s+(\d{3}\.\d)')
matches = data_pattern.findall(data)

In [31]:
columns = ['YEAR', 'DOY', 'HR', 'Kp_index', 'R', 'Dst_index', 'ap_index', 'f10.7_index']
df = pd.DataFrame(matches, columns=columns)

In [32]:
df = df.astype({
    'YEAR': int,
    'DOY': int,
    'HR': int,
    'Kp_index': int,
    'R': int,
    'Dst_index': int,
    'ap_index': int,
    'f10.7_index': float
})


In [33]:
df.head()

,YEAR,DOY,HR,Kp_index,R,Dst_index,ap_index,f10.7_index
0,2024,191,0,20,999,7,7,184.2
1,2024,191,1,20,999,9,7,184.2
2,2024,191,2,20,999,10,7,184.2
3,2024,191,3,17,999,7,6,184.2
4,2024,191,4,17,999,4,6,184.2


In [34]:
df.shape

(24, 8)

In [35]:
new_df = df[['YEAR', 'R', 'f10.7_index']] 
print(new_df.head())

   YEAR    R  f10.7_index
0  2024  999        184.2
1  2024  999        184.2
2  2024  999        184.2
3  2024  999        184.2
4  2024  999        184.2


In [36]:
def df_to_X_y2(df, window_size=6):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i:i + window_size]]
        X.append(row)
        label = df_as_np[i + window_size][0]  # Adjust the label indexing as necessary
        y.append(label)
    return np.array(X), np.array(y)

NEW_WINDOW_SIZE = 6
X_new, y_new = df_to_X_y2(new_df, NEW_WINDOW_SIZE)




In [37]:
scaler = MinMaxScaler()

# Fit and transform the new data
X_new_normalized = scaler.fit_transform(X_new.reshape(-1, X_new.shape[-1])).reshape(X_new.shape)
y_new_normalized = scaler.fit_transform(y_new.reshape(-1, y_new.shape[-1])).reshape(y_new.shape)

print(X_new_normalized.shape, y_new_normalized.shape)

(18, 6, 3) (18,)


In [39]:
# Load the trained model
model4 = load_model('models\model4\m4.keras')


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 6, 4], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_13'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [27]:
# Predicting the values
predictions = model4.predict(X_new_normalized)

# Inverse transform the predictions if necessary
predictions = scaler.inverse_transform(predictions)

print(predictions)


AttributeError: module 'models.model4' has no attribute 'predict'